<h1 style="font-size:40px;"><center>Exercise IV: Classification
    
</center></h1>

For this exercise you are given a classification problem with a fixed training-, validation- and test dataset. The data is the Japanse vowels dataset described in the first cell. Your task is to do model selection, coming up with your optimal MLP architecture together with the hyperparameters you used. We do not provide any python code for this question, only the small part that reads the data (next code cell). There is an additional code cell that can be used to print the confusion matrix for the test data (see comments within that cell). (You can modify this function if you want to compute this matrix for both training and validation.)

# Data
## Spiral data
This is the "famous" spiral dataset that consists of two 2-D spirals, one for each class. The perfect classification boundary is also a spiral. The cell "PlotData" will plot this dataset.

## Japanese vowels dataset
*This data set is taken from the UCI Machine Learning Repository [https://archive.ics.uci.edu/ml/datasets/Japanese+Vowels].* In short, nine male speakers uttered two Japanese vowels /ae/ successively. For each utterance, a discrete times series was produced where each time point consists of 12 (LPC cepstrum) coefficients. The length of each time series was between 7-29. 
Here we treat each point of the time series as a feature (12 inputs). In total we have 9961
data points which then has been divided into 4274 for training, 2275 for validation and 3412 for test. The original data files are provided as *ae.train* and *ae.test*. The task is now based on a single sample value of one of the speakers, determine which speaker it was. This is, in summary, a 9-class classification problem with 12 input values for each case.
# Tasks

## Task 1
**TODO:** Present an MLP with associated hyperparameters that maximizes the validation performance and give the test performance you obtained.

**Hint 1:** 
Remember to check if input data needs to be normalized. See the "Ex3" cell how this was done (in that case for the target data).

**Hint 2:** 
This problem is a 9-class classification problem, meaning that you should use a specific output activation function (*out_act_fun*) and a specific loss/error function (*cost_fun*).

**Hint 3:**
The function *stats_class* function does not work here since it is designed for binary classification problems.


# Code

The following code allows us to edit imported files without restarting the kernel for the notebook

In [1]:
%load_ext autoreload
%autoreload 2

# Hacky solution to access the global utils package
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('..')))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl

from config import LabConfig
from dataset import MLPData
from DL_labs.utils.model import Model
from DL_labs.utils.progressbar import LitProgressBar
from DL_labs.utils.model import Model
from torch.utils.data import TensorDataset, DataLoader
from DL_labs.utils import (
    plot,
    progressbar
) 

/home/stud/f/fremar16/miniconda3/envs/DL-Labs/lib/python3.9/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


# Defining the MLP model

This cell defines the MLP model. There are a number of parameters that is needed to 
define a model. Here is a list of them: **Note:** They can all be specified when you call
this function in later cells. The ones specified in this cell are the default values.


In [3]:
class MLP(torch.nn.Module):
    def __init__(self, 
                inp_dim=None,         
                hidden_nodes=1,                      # number of nodes in hidden layer
                num_out=None,
                **kwargs
            ):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(inp_dim, hidden_nodes)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_nodes, num_out)


    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        return torch.sigmoid(output)

## Define a function that allow us to convert numpy to pytorch DataLoader

In [4]:
def numpy2Dataloader(x,y, batch_size=25, num_workers=10,**kwargs):
    return DataLoader(
        TensorDataset(
            torch.from_numpy(x).float(), 
            torch.from_numpy(y).unsqueeze(1).float()
        ),
        batch_size=batch_size,
        num_workers=num_workers,
        **kwargs
    )

## Load dataset

In [5]:
x_train, y_train, x_val, y_val, x_test, y_test = MLPData.vowels()
num_classes = 9

print('train input size: ',x_train.shape, 'train target size: ',y_train.shape)
print('val   input size: ',x_val.shape, 'val   target size: ',y_val.shape)
print('test  input size: ',x_test.shape, 'test  target size: ',y_test.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'ae.train'

## Configuration
Setup our local config that should be used for the trainer.

In [ ]:
config = {
    'max_epochs':4,
    'model_params':{
        'inp_dim':x.shape[1],         
        'hidden_nodes':1,   # activation functions for the hidden layer
        'num_out':1 # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'criterion':torch.nn.BCELoss(), # error function
    'optimizer':{
        "type":torch.optim.Adam,
        "args":{
            "lr":0.005,
        }
    }
}

## Training

Lastly, put everything together and call on the trainers fit method. 

In [ ]:
model = Model(MLP(**config["model_params"]),**config)

trainer = pl.Trainer(
            max_epochs=config['max_epochs'], 
            gpus=cfg.GPU,
            logger=pl.loggers.TensorBoardLogger(save_dir=cfg.TENSORBORD_DIR),
            callbacks=[LitProgressBar()],
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,   
        )
trainer.fit(
    model, 
    train_dataloader=train_loader
);

## Evaluation

In [ ]:
"""
This cell can be used to present the classification results. 
It assumes you trained model has the name 'model_vowels'. 
It shows the test data results, but it can be changed to show
the training/validation.
"""
print('\n','#'*10,'Result for {} Data'.format('Test'), '#'*10, '\n')

y_pred = model_vowels.predict(x_test, verbose=0 )
print('log_loss:   ', log_loss(y_test, y_pred, eps=1e-15))

y_true = y_test.argmax(axis=1)
y_pred = y_pred.argmax(axis=1)
print('accuracy:   ',(y_pred==y_true).mean(), '\n')

target_names = ['class {}'.format(i+1) for i in range(9)]
print(classification_report(y_true, y_pred, target_names=target_names))

confuTst = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm           = confuTst, 
                      normalize    = False,
                      target_names = ['1', '2', '3', '4', '5', '6','7', '8','9'],
                      title        = "Confusion Matrix: Test data")



## Task 2
For this last exercise the task is to train a binary classifier for the spiral problem. The aim is to get *zero* classification error for the training data (there is no test data) with as small as possible model, in terms of the number of trainable weights. Also plot the boundary to see if it resembles a spriral. To pass this question you should at least try!

**TODO:** Train a classifier for the spiral problem with the aim of zero classification error with as small as possible model.

In [ ]:
# seed = 0 means random, seed > 0 means fixed

# Generate training data
x_train, d_train = MLPData.spiral()
x_train = x_train / 5

# Define the network, cost function and minimization method,
# train and evaluate the result
